In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povms.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povms.post_processor.povm_post_processor import POVMPostprocessor
from povms.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

# Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 2
seed = 13
qc_random = random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed)
qc_random.draw()

┌──────┐┌──────────────────────────┐             
q_0: ┤0     ├┤1                         ├─■───────────
     │  Ecr ││  (XX-YY)(1.6427,0.48506) │ │ZZ(3.8566) 
q_1: ┤1     ├┤0                         ├─■───────────
     └──────┘└──────────────────────────┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exact_state = Statevector(np.array(Operator(qc_random).data[:,0]).squeeze())
exp_val = [exact_state.expectation_value(obs) for obs in set_observables]

# Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the shot budget.
cs_shots = 4096

# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(cs_implementation, qc_random, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:156: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


## Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [7]:
cs_result = cs_job.result()
print(f"POVM: {cs_result.povm}")
print(f"Counts: {cs_result.get_counts()}")

POVM: <povms.library.pm_sim_implementation.ClassicalShadows object at 0x118f58130>
Counts: {(0, 5): 65, (1, 5): 57, (1, 4): 274, (0, 4): 65, (2, 1): 63, (2, 0): 239, (3, 0): 101, (3, 1): 55, (4, 0): 348, (4, 1): 56, (5, 0): 22, (5, 1): 63, (1, 1): 101, (1, 0): 221, (0, 0): 127, (3, 2): 167, (2, 3): 111, (2, 2): 151, (3, 3): 1, (5, 4): 73, (4, 4): 276, (4, 5): 111, (5, 5): 4, (4, 2): 268, (4, 3): 101, (5, 3): 29, (5, 2): 31, (2, 4): 269, (3, 4): 104, (3, 5): 75, (2, 5): 23, (1, 2): 271, (1, 3): 56, (0, 3): 64, (0, 2): 54}


We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [8]:
cs_postprocessor = POVMPostprocessor(cs_result)
cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]

In [9]:
n = int(np.ceil(np.log10(cs_shots)))
print('Exact        CS')
print(f'             ({cs_shots} shots)')
for i in range(len(set_observables)):
    print(f'{np.real(exp_val[i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}')

Exact        CS
             (4096 shots)
 4.041e-01      4.093e-01
 2.614e+00      2.655e+00
 5.072e-01      5.535e-01
-8.583e-01     -9.000e-01
-1.899e-01     -2.214e-01
 3.017e-01      3.402e-01
 4.705e-01      4.404e-01
 4.975e-01      5.638e-01
 1.980e+00      2.013e+00
 6.090e-01      6.483e-01


# PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [10]:
# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[1.2,0.0, 0.1,3.14,0.7,0.3]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.7,0.15,0.15]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles)
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pmsim_implementation, qc_random, shots=pmsim_shots)

## Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [11]:
pmsim_result = pmsim_job.result()
print(f"Counts: {pmsim_result.get_counts()}")

pmsim_postprocessor = POVMPostprocessor(pmsim_result)
pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]

Counts: {(3, 0): 332, (2, 0): 412, (3, 1): 2, (2, 1): 189, (3, 2): 60, (3, 3): 32, (2, 2): 104, (2, 3): 25, (1, 4): 130, (1, 5): 17, (0, 4): 48, (0, 5): 6, (1, 0): 628, (0, 1): 70, (0, 0): 167, (1, 1): 123, (4, 2): 156, (4, 3): 36, (5, 2): 9, (5, 3): 24, (0, 2): 64, (1, 3): 53, (1, 2): 99, (4, 0): 653, (4, 1): 98, (5, 1): 87, (5, 0): 70, (5, 5): 13, (5, 4): 7, (4, 4): 178, (4, 5): 4, (2, 4): 113, (3, 4): 67, (2, 5): 15, (3, 5): 5}


In [12]:
n = int(np.ceil(np.log10(cs_shots)))
print(f'Exact        {"CS":<{8+n}}   PM-Sim')
print(f'             ({cs_shots} shots)   ({pmsim_shots} shots)')
for i in range(len(set_observables)):
    print(f'{np.real(exp_val[i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}')

Exact        CS             PM-Sim
             (4096 shots)   (4096 shots)
 4.041e-01      4.093e-01      4.586e-01
 2.614e+00      2.655e+00      2.756e+00
 5.072e-01      5.535e-01      5.974e-01
-8.583e-01     -9.000e-01     -1.036e+00
-1.899e-01     -2.214e-01     -3.270e-01
 3.017e-01      3.402e-01      5.928e-01
 4.705e-01      4.404e-01      1.990e-01
 4.975e-01      5.638e-01      6.202e-01
 1.980e+00      2.013e+00      1.675e+00
 6.090e-01      6.483e-01      8.681e-01
